# Excel Quality Pipeline – Discovery Notebook

## Objective
Build a reproducible data pipeline that produces a single cleaned
quality dataset from multiple Excel inputs.

This notebook is for exploration and validation only.
Final logic will be migrated into the `src/` module.


In [321]:
import pandas as pd
pd.__version__


'2.3.3'

In [322]:
file_path = "../data/raw/PATIENT OPD VISITS- from MCC & Email.xlsx"
df = pd.read_excel(file_path)
df.shape

(4204, 18)

In [323]:
header = df.columns.tolist()

In [324]:
print(header)

['MRN', 'PatEngName', 'pat_birthdate', 'regtime', 'VISIT_DATETIME', 'ARRIVETIME', 'STARTTIME', 'VISIT_TYPE', 'PORTAL STATUS', 'VISIT STATUS', 'BOOKING_TYPE', 'clinic_key', 'CLINIC_NAME', 'MRP', 'JOB TITLE', 'SPECIALTY', 'Added_by', 'addtime']


In [325]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4204 entries, 0 to 4203
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   MRN             4204 non-null   int64         
 1   PatEngName      4094 non-null   object        
 2   pat_birthdate   4047 non-null   datetime64[ns]
 3   regtime         4204 non-null   datetime64[ns]
 4   VISIT_DATETIME  4047 non-null   datetime64[ns]
 5   ARRIVETIME      3314 non-null   datetime64[ns]
 6   STARTTIME       3208 non-null   datetime64[ns]
 7   VISIT_TYPE      4204 non-null   object        
 8   PORTAL STATUS   4047 non-null   object        
 9   VISIT STATUS    4047 non-null   object        
 10  BOOKING_TYPE    4047 non-null   object        
 11  clinic_key      4047 non-null   float64       
 12  CLINIC_NAME     4204 non-null   object        
 13  MRP             4204 non-null   object        
 14  JOB TITLE       4047 non-null   object        
 15  SPEC

In [326]:
# Drop uneeded col
cols_to_drop = [
    "regtime",
    "ARRIVETIME",
    "STARTTIME",
    "PORTAL STATUS",
    "clinic_key",
    "JOB TITLE",
    "SPECIALTY",
    "Added_by",
    "addtime",
]

df = df.drop(columns=cols_to_drop)


In [327]:
df.columns

Index(['MRN', 'PatEngName', 'pat_birthdate', 'VISIT_DATETIME', 'VISIT_TYPE',
       'VISIT STATUS', 'BOOKING_TYPE', 'CLINIC_NAME', 'MRP'],
      dtype='object')

In [328]:
# Normalize column names
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
)
df.columns

Index(['mrn', 'patengname', 'pat_birthdate', 'visit_datetime', 'visit_type',
       'visit_status', 'booking_type', 'clinic_name', 'mrp'],
      dtype='object')

In [329]:
df['visit_datetime'].dtype

dtype('<M8[ns]')

In [330]:
# change col visit_datetime to datetime DD/MM/YYYY

df["visit_datetime"] = pd.to_datetime(
    df["visit_datetime"],
    errors="coerce"
).dt.date


In [331]:
df["visit_datetime"].dtype
type(df.loc[0, "visit_datetime"])


datetime.date

In [332]:
# Adding the new col

new_cols = [
    "liver_r_d",
    "liver_status",
    "kidney_recipient",
    "kidney_donor",
]

for col in new_cols:
    df[col] = None

df.columns

Index(['mrn', 'patengname', 'pat_birthdate', 'visit_datetime', 'visit_type',
       'visit_status', 'booking_type', 'clinic_name', 'mrp', 'liver_r_d',
       'liver_status', 'kidney_recipient', 'kidney_donor'],
      dtype='object')

In [333]:
df.head()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor
0,218455,NORAH BAHIAS ALI ZARAB,1987-04-16,2025-12-23,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,None,None,None,None
1,118631,ZAHRA MAHDI ALI ALAWAZIM,2003-02-24,2025-12-10,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,None,None,None,None
2,219640,SULTANAH HAMAD ABDULLAH ALHASSAN,1959-01-10,2025-12-08,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,None,None,None,None
3,220961,SEHAM MOHAMMED KHAMEES ALHASSAN,1969-06-03,2025-12-10,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,None,None,None,None
4,220533,ABDULAZIZ ADNAN SALEH ALHADAD,1978-09-30,2025-12-21,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,None,None,None,None


In [334]:
df['visit_status'].value_counts()

visit_status
STARTED    3208
NO SHOW     733
ARRIVED     106
Name: count, dtype: int64

In [335]:
# Remove NO SHOW visit_status

df = df[df["visit_status"] != "NO SHOW"]


In [336]:
df['visit_status'].value_counts()

visit_status
STARTED    3208
ARRIVED     106
Name: count, dtype: int64

In [337]:
# Fill liver columns using Liver Registry (ONLY)
# For each MRN in df:
# Look it up in Liver Registry
# If found:
###### Rec_Don → liver_r_d
###### Current Status1 → liver_status
# If not found:
##### leave as NaN

liver_df = pd.read_excel("../data/raw/Liver Registry - 1 JAN.xlsx")
liver_df.columns = (liver_df.columns.str.strip().str.lower().str.replace(" ", "_"))



In [338]:
liver_df.columns.tolist()



['r_mrn',
 'id_number',
 'patient_name',
 'mobile1_0',
 'payment_0',
 'rec_don',
 'nationality',
 'city_of_residence',
 'specify_other',
 'age_group',
 'age',
 'blood_group',
 'gender',
 'date_open_file_0',
 'current_status1',
 'patient_type_1',
 'initial_tx_assessment_date',
 'workup_start_date',
 'workup_completed_date',
 'date_added_to_waiting_list',
 'surgery_date',
 'tx_type',
 'transplant_place',
 'comment_0',
 'closefile_r',
 'closefiledate_r',
 'closefilereason_r',
 'assigned_consultant_0',
 'assigned_surgeon_r',
 'primary_disease']

In [339]:
df = df.merge(
    liver_df[["r_mrn", "rec_don", "current_status1"]],
    left_on="mrn",
    right_on="r_mrn",
    how="left"
)


In [340]:
df["liver_r_d"] = df["rec_don"]
df["liver_status"] = df["current_status1"]
df.tail()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,r_mrn,rec_don,current_status1
3466,203844,Abdullah,NaT,NaT,Follow up,NaN,NaN,Psychology,Alia Al Enazi,Recipient,Post Tx,None,None,203844,Recipient,Post Tx
3467,221651,MOHAMMED,NaT,NaT,New,NaN,NaN,Psychology,Alia Al Enazi,NaN,NaN,None,None,NaN,NaN,NaN
3468,221730,Abdulqawi,NaT,NaT,New,NaN,NaN,Psychology,Alia Al Enazi,NaN,NaN,None,None,NaN,NaN,NaN
3469,40991,SHATHA,NaT,NaT,New,NaN,NaN,Psychology,Alia Al Enazi,NaN,NaN,None,None,NaN,NaN,NaN
3470,107061,ABDULLAH,NaT,NaT,New,NaN,NaN,Psychology,Alia Al Enazi,NaN,NaN,None,None,NaN,NaN,NaN


In [341]:
df = df.drop(columns=["r_mrn", "rec_don", "current_status1"])


In [342]:
df[df["liver_r_d"].notna()]


,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor
12,104431,SAIF RAJA H ALHAJRI,1976-06-28,2025-12-10,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,Recipient,Hepatology patient,None,None
20,220797,BASHAYER FAWAZ ALSAEGR,1992-04-17,2025-12-10,Walk-In,STARTED,WALK-IN,TRANSPLANT SURGERY,Mohammed Saad Ali AL-Qahtani,Donor,Pre Tx ready for transplant,None,None
21,220503,ALHASSAN ABDULRAHIM ABDULLAH ALZAHRANI,1999-08-20,2025-12-10,Walk-In,STARTED,WALK-IN,TRANSPLANT SURGERY,Mohammed Saad Ali AL-Qahtani,Donor,Post Tx,None,None
22,220468,SAAD SALEM M ALMAHRI,1986-03-17,2025-12-10,Walk-In,STARTED,WALK-IN,TRANSPLANT SURGERY,Mohammed Saad Ali AL-Qahtani,Donor,Post Tx,None,None
23,220993,IBRAHIM ABDULKARIM IBRAHIM ALHABIB,1993-09-13,2025-12-10,Walk-In,STARTED,WALK-IN,TRANSPLANT SURGERY,Mohammed Saad Ali AL-Qahtani,Donor,Pre Tx ready for transplant,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3454,221422,MESHARI,NaT,NaT,New,NaN,NaN,Psychology,Alia Al Enazi,Donor,Pre Tx on workups,None,None
3455,221756,FAHAD,NaT,NaT,New,NaN,NaN,Psychology,Alia Al Enazi,Donor,Pre Tx on workups,None,None
3456,218992,HISSAH,NaT,NaT,New,NaN,NaN,Psychology,Alia Al Enazi,Recipient,Pre Tx on workups,None,None
3461,221468,WADI,NaT,NaT,New,NaN,NaN,Psychology,Alia Al Enazi,Donor,Pre Tx on workups,None,None


In [343]:
# Fill kidney_recipient using Kidney Recipient registry (ONLY)
# For each mrn in the main df:
# Look it up in Kidney Recipient - 1 JAN.xlsx

# If found:
###### copy Current Status1 → kidney_recipient
# If not found:
###### leave as NaN

kidney_rec_df = pd.read_excel("../data/raw/Kidney Recipient - 1 JAN.xlsx")
kidney_rec_df.columns = (kidney_rec_df.columns.str.strip().str.lower().str.replace(" ", "_"))


In [344]:
df = df.merge(
    kidney_rec_df[["r_mrn", "current_status1"]],
    left_on="mrn",
    right_on="r_mrn",
    how="left"
)
df.head()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,r_mrn,current_status1
0,218455,NORAH BAHIAS ALI ZARAB,1987-04-16,2025-12-23,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,None,None,NaN,NaN
1,118631,ZAHRA MAHDI ALI ALAWAZIM,2003-02-24,2025-12-10,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,None,None,NaN,NaN
2,219640,SULTANAH HAMAD ABDULLAH ALHASSAN,1959-01-10,2025-12-08,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,None,None,NaN,NaN
3,220961,SEHAM MOHAMMED KHAMEES ALHASSAN,1969-06-03,2025-12-10,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,None,None,NaN,NaN
4,220533,ABDULAZIZ ADNAN SALEH ALHADAD,1978-09-30,2025-12-21,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,None,None,NaN,NaN


In [345]:
df["kidney_recipient"] = df["current_status1"]
df = df.drop(columns=["r_mrn", "current_status1"])



In [346]:
df.tail()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor
3466,203844,Abdullah,NaT,NaT,Follow up,NaN,NaN,Psychology,Alia Al Enazi,Recipient,Post Tx,Nephrology patient,None
3467,221651,MOHAMMED,NaT,NaT,New,NaN,NaN,Psychology,Alia Al Enazi,NaN,NaN,NaN,None
3468,221730,Abdulqawi,NaT,NaT,New,NaN,NaN,Psychology,Alia Al Enazi,NaN,NaN,NaN,None
3469,40991,SHATHA,NaT,NaT,New,NaN,NaN,Psychology,Alia Al Enazi,NaN,NaN,NaN,None
3470,107061,ABDULLAH,NaT,NaT,New,NaN,NaN,Psychology,Alia Al Enazi,NaN,NaN,Post LD Tx,None


In [347]:
# Fill kidney_donor
# For each mrn in the main df:
# Match it to d_mrn in Kidney Donor - 1 JAN.xlsx
# Copy current_status1_d → kidney_donor
# If no match → leave NaN

kidney_donor_df = pd.read_excel("../data/raw/Kidney Donor - 1 JAN.xlsx")

kidney_donor_df.columns = (
    kidney_donor_df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
)

df = df.merge(
    kidney_donor_df[["d_mrn", "current_status1_d"]],
    left_on="mrn",
    right_on="d_mrn",
    how="left"
)

df["kidney_donor"] = df["current_status1_d"]

df = df.drop(columns=["d_mrn", "current_status1_d"])


In [348]:
df.head()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor
0,218455,NORAH BAHIAS ALI ZARAB,1987-04-16,2025-12-23,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,NaN,NaN
1,118631,ZAHRA MAHDI ALI ALAWAZIM,2003-02-24,2025-12-10,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,NaN,NaN
2,219640,SULTANAH HAMAD ABDULLAH ALHASSAN,1959-01-10,2025-12-08,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,NaN,NaN
3,220961,SEHAM MOHAMMED KHAMEES ALHASSAN,1969-06-03,2025-12-10,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,NaN,NaN
4,220533,ABDULAZIZ ADNAN SALEH ALHADAD,1978-09-30,2025-12-21,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,NaN,NaN


In [349]:
df['clinic_name'].value_counts()

clinic_name
Anesthesia                                     840
Clinical Nutrition                             556
Adult Post Renal Transplant                    500
TRANSPLANT SURGERY                             458
Adult Post Kidney Transplant Virtual Clinic    264
Adult Pre Renal Transplant                     133
ADULT POST LIVER TRANSPLANT                    131
Social Work                                    110
Pediatric Post Renal Transplant                102
Adult Pre Liver Transplant                      97
Pediatric Liver Transplant                      66
Transplant Dressing (procedure)                 54
Psychology                                      47
Adult Post Liver Transplant Virtual             42
Adult Pre Renal Transplant Virtual Clinic       37
Pre anesthesia liver transplant                 15
Pediatric Pre Renal Transplant                  11
Adult Pre Liver Transplant Virtual               5
Transplant Cystoscopy Clinic                     3
Name: count, dtype:

In [350]:
# Remove visits based on clinic + transplant status (ONLY)

# Remove row if ALL of the following are true:

# 1-clinic_name is Anesthesia OR Clinical Nutrition
# 2- liver_r_d is NaN
# 3-kidney_recipient is NaN
# 4-kidney_donor is NaN

mask = (
    df["clinic_name"].isin(["Anesthesia", "Clinical Nutrition"])
    & df["liver_r_d"].isna()
    & df["kidney_recipient"].isna()
    & df["kidney_donor"].isna()
)

df = df[~mask]


In [351]:
df[
    df["clinic_name"].isin(["Anesthesia", "Clinical Nutrition"])
][["liver_r_d", "kidney_recipient", "kidney_donor"]].isna().all(axis=1).sum()


np.int64(0)

In [352]:
df.head()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor
7,174153,AYSH ABDO HAMAD HADIDI,1955-02-23,2025-12-23,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,SUMAYAH ABDULAZIZ MOHAMMED ASKANDARANI,NaN,NaN,Post LD Tx,NaN
9,77521,AMAL SULAIMAN OUDAH ALHAWITI,1986-09-28,2025-12-17,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,Post DD Tx,NaN
12,104431,SAIF RAJA H ALHAJRI,1976-06-28,2025-12-10,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,Recipient,Hepatology patient,Post DD Tx,NaN
13,101282,NOUF SAUD MOHAMMED ALDAWSARI,1982-11-29,2025-12-14,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,Post LD Tx,NaN
17,6364,MOHAMMED BEN ALI BIN SULAIMAN AL JAFAR,1982-08-20,2025-12-21,FOLLOW UP,STARTED,NORMAL BOOKING,TRANSPLANT SURGERY,MAHMOUD ALI MAHMOUD OBEID,NaN,NaN,Post LD Tx,NaN


In [353]:
df[df[["liver_r_d", "kidney_recipient", "kidney_donor"]].isna().all(axis=1)]

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor
530,13459,SHAIKHAH MOHAMMED A ALSAMHAN,1940-08-04,2025-12-15,NEW,STARTED,NORMAL BOOKING,Adult Pre Liver Transplant,Eyad Mohammad Osman Elhassan Gadour,NaN,NaN,NaN,NaN
1166,221813,MOHAMMAD MOQBIL ABDULLAH ALQARAWI,1959-01-10,2025-12-28,NEW,STARTED,NORMAL BOOKING,Adult Pre Liver Transplant,HADI MOHAMMED SHOEI KURIRY,NaN,NaN,NaN,NaN
1169,218567,REEM ABDULHAKIM HAMAD ALKHALDI,2010-07-14,2025-12-02,Walk-In,STARTED,WALK-IN,Pediatric Liver Transplant,Razan Monther Abdulmajied Bader,NaN,NaN,NaN,NaN
2335,8909,BANEEN ABDULLAH MOHAMMED ALSAIHATI,1986-05-17,2025-12-29,FOLLOW UP,STARTED,NORMAL BOOKING,TRANSPLANT SURGERY,MAHMOUD ALI MAHMOUD OBEID,NaN,NaN,NaN,NaN
2744,200160,AHMED MOHAMED ALY AHMED ZIDAN,1981-07-25,2025-12-31,CHART CHECK,STARTED,WALK-IN,TRANSPLANT SURGERY,AHMED MOHAMED ALY AHMED ZIDAN,NaN,NaN,NaN,NaN
2756,6097,REHAB AHMED GABER ABDALL,1978-06-10,2025-12-28,CHART CHECK,STARTED,WALK-IN,TRANSPLANT SURGERY,AHMED MOHAMED ALY AHMED ZIDAN,NaN,NaN,NaN,NaN
3018,220426,SALMAN OMAR JAFAR ALOTAIBI,1986-09-14,2025-12-09,Walk-In,STARTED,WALK-IN,TRANSPLANT SURGERY,Mohammed Saad Ali AL-Qahtani,NaN,NaN,NaN,NaN
3041,200362,NISY MATHEWS,1998-08-15,2025-12-16,Walk-In,STARTED,WALK-IN,Adult Pre Renal Transplant,KHALID AHMEDH BELEED AKKARI,NaN,NaN,NaN,NaN
3170,210421,Devika Reghu,1996-05-04,2025-12-29,Walk-In,STARTED,WALK-IN,TRANSPLANT SURGERY,MAHMOUD ALI MAHMOUD OBEID,NaN,NaN,NaN,NaN


In [354]:
# Add need_review flag
df["need_review"] = "no"
review_mask = (
    df[["liver_r_d", "kidney_recipient", "kidney_donor"]]
    .isna()
    .all(axis=1)
)

df.loc[review_mask, "need_review"] = "yes"


In [355]:
df['liver_r_d'].value_counts()


liver_r_d
Recipient    547
Donor        125
Name: count, dtype: int64

In [356]:
# Create and fill r_d (LOCKED LOGIC)

# We look at three columns:
#liver_r_d, kidney_recipient, kidney_donor
# A) If only one of the three is non-NaN:
# Only liver_r_d filled → r_d = "Liver " + liver_r_d
# Only kidney_recipient filled → r_d = kidney_recipient
# Only kidney_donor filled → r_d = kidney_donor
# B) If liver_r_d is non-NaN AND (either kidney_recipient OR kidney_donor is non-NaN):
# Use clinic_name keyword:
# If clinic_name contains renal or kidney → r_d becomes the non-NaN kidney value (recipient preferred, else donor)
# If clinic_name contains liver → r_d = "Liver " + liver_r_d

# Vectorized version
mask_liver_only = df['liver_r_d'].notna() & df['kidney_recipient'].isna() & df['kidney_donor'].isna()
mask_k_recip_only = df['kidney_recipient'].notna() & df['liver_r_d'].isna() & df['kidney_donor'].isna()
mask_k_donor_only = df['kidney_donor'].notna() & df['liver_r_d'].isna() & df['kidney_recipient'].isna()

df['r_d'] = np.select(
    [
        mask_liver_only,
        mask_k_recip_only,
        mask_k_donor_only,
    ],
    [
        "Liver " + df['liver_r_d'].astype(str),
        df['kidney_recipient'],
        df['kidney_donor']
    ],
    default=np.nan
)

# Now handle mixed cases (where liver + at least one kidney is present)
mixed_mask = (
    df['liver_r_d'].notna() &
    (df['kidney_recipient'].notna() | df['kidney_donor'].notna()) &
    (df['r_d'].isna())  # still null after previous step
)

clinic_lower = df['clinic_name'].fillna('').str.lower()

renal_clinic = clinic_lower.str.contains('renal|kidney')
liver_clinic = clinic_lower.str.contains('liver')

df.loc[mixed_mask & renal_clinic & ~liver_clinic, 'r_d'] = \
    df.loc[mixed_mask & renal_clinic & ~liver_clinic, 'kidney_recipient'].fillna(
        df.loc[mixed_mask & renal_clinic & ~liver_clinic, 'kidney_donor']
    )

df.loc[mixed_mask & liver_clinic & ~renal_clinic, 'r_d'] = \
    "Liver " + df.loc[mixed_mask & liver_clinic & ~renal_clinic, 'liver_r_d'].astype(str)




In [357]:
df.head()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,need_review,r_d
7,174153,AYSH ABDO HAMAD HADIDI,1955-02-23,2025-12-23,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,SUMAYAH ABDULAZIZ MOHAMMED ASKANDARANI,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx
9,77521,AMAL SULAIMAN OUDAH ALHAWITI,1986-09-28,2025-12-17,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,Post DD Tx,NaN,no,Post DD Tx
12,104431,SAIF RAJA H ALHAJRI,1976-06-28,2025-12-10,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,Recipient,Hepatology patient,Post DD Tx,NaN,no,NaN
13,101282,NOUF SAUD MOHAMMED ALDAWSARI,1982-11-29,2025-12-14,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx
17,6364,MOHAMMED BEN ALI BIN SULAIMAN AL JAFAR,1982-08-20,2025-12-21,FOLLOW UP,STARTED,NORMAL BOOKING,TRANSPLANT SURGERY,MAHMOUD ALI MAHMOUD OBEID,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx


In [358]:
df['liver_status'].value_counts()

liver_status
Post Tx                        366
Hepatology patient             111
Pre Tx on workups               82
New patient                     47
Pre Tx ready for transplant     24
OPD Hepatology patient          16
Pre Tx on DDWL                  16
Pre Tx Ready for transplant      6
Hepatitis C patient              2
Pre Tx on hold                   2
Name: count, dtype: int64

In [359]:
# ambiguous_mask = (
#     df['liver_r_d'].notna() &
#     (df['kidney_recipient'].notna() | df['kidney_donor'].notna()) &
#     df['r_d'].isna()
# )

# len(df.loc[ambiguous_mask])

# come to this later at the end to see how many should be assigned as need review
df.head(30)

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,need_review,r_d
7,174153,AYSH ABDO HAMAD HADIDI,1955-02-23,2025-12-23,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,SUMAYAH ABDULAZIZ MOHAMMED ASKANDARANI,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx
9,77521,AMAL SULAIMAN OUDAH ALHAWITI,1986-09-28,2025-12-17,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,Post DD Tx,NaN,no,Post DD Tx
12,104431,SAIF RAJA H ALHAJRI,1976-06-28,2025-12-10,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,Recipient,Hepatology patient,Post DD Tx,NaN,no,NaN
13,101282,NOUF SAUD MOHAMMED ALDAWSARI,1982-11-29,2025-12-14,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx
17,6364,MOHAMMED BEN ALI BIN SULAIMAN AL JAFAR,1982-08-20,2025-12-21,FOLLOW UP,STARTED,NORMAL BOOKING,TRANSPLANT SURGERY,MAHMOUD ALI MAHMOUD OBEID,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx
18,66898,AMNA AHMED ABDULGAFOR AL SHEIKH,1984-01-04,2025-12-10,FOLLOW UP,ARRIVED,NORMAL BOOKING,TRANSPLANT SURGERY,MAHMOUD ALI MAHMOUD OBEID,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx
19,208799,FARZANAH SAEED A ALI,1963-07-01,2025-12-15,FOLLOW UP,STARTED,NORMAL BOOKING,TRANSPLANT SURGERY,MAHMOUD ALI MAHMOUD OBEID,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx
20,220797,BASHAYER FAWAZ ALSAEGR,1992-04-17,2025-12-10,Walk-In,STARTED,WALK-IN,TRANSPLANT SURGERY,Mohammed Saad Ali AL-Qahtani,Donor,Pre Tx ready for transplant,NaN,NaN,no,Liver Donor
21,220503,ALHASSAN ABDULRAHIM ABDULLAH ALZAHRANI,1999-08-20,2025-12-10,Walk-In,STARTED,WALK-IN,TRANSPLANT SURGERY,Mohammed Saad Ali AL-Qahtani,Donor,Post Tx,NaN,NaN,no,Liver Donor
22,220468,SAAD SALEM M ALMAHRI,1986-03-17,2025-12-10,Walk-In,STARTED,WALK-IN,TRANSPLANT SURGERY,Mohammed Saad Ali AL-Qahtani,Donor,Post Tx,NaN,NaN,no,Liver Donor


In [360]:
df['r_d'].value_counts()

r_d
Post LD Tx                                        911
Liver Recipient                                   450
Pre Tx on workups                                 266
Post living donor nephrectomy                     158
Liver Donor                                       122
Post DD Tx                                         98
Waiting for first appointment                      50
Listed on DDWL                                     40
Pre Tx Ready for committee                         33
Pre Tx Discussed & ready for donor nephrectomy     30
Nephrology patient                                 21
Initial visit done, No LD, suspended for DDWL      20
Pre Tx Discussed & Ready for LD Tx                 18
Listed on DDWL & Ready for LD Tx                   13
Pre Tx Discussed with pending issues                9
Pre Tx on hold                                      9
Listed on DDWL - Inactive                           3
Name: count, dtype: int64

In [361]:
df["patient_type"] = None
df["transplant_date"] = None
df["days"] = None


In [362]:
df.head(1)

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,need_review,r_d,patient_type,transplant_date,days
7,174153,AYSH ABDO HAMAD HADIDI,1955-02-23,2025-12-23,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,SUMAYAH ABDULAZIZ MOHAMMED ASKANDARANI,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,None,None,None


In [363]:
df['patient_type'] = df['liver_status'].combine_first(
    df['kidney_recipient']
).combine_first(
    df['kidney_donor']
)

In [364]:
# if patient_type is NA i think need_revuew = yes

In [365]:
# Create lookup dictionaries: MRN → Surgery Date
# Using the actual column names from your files

liver_dict = (
    liver_df[['r_mrn', 'surgery_date']]
    .dropna(subset=['r_mrn'])
    .set_index('r_mrn')['surgery_date']
    .to_dict()
)

recip_dict = (
    kidney_rec_df[['r_mrn', 'surgery_date']]           # ← using kidney_rec_df (your naming)
    .dropna(subset=['r_mrn'])
    .set_index('r_mrn')['surgery_date']
    .to_dict()
)

donor_dict = (
    kidney_donor_df[['d_mrn', 'surgery_date_d']]
    .dropna(subset=['d_mrn'])
    .set_index('d_mrn')['surgery_date_d']
    .to_dict()
)

# Populate transplant_date with this priority order:
# 1. Liver
# 2. Kidney Recipient
# 3. Kidney Donor
df['transplant_date'] = (
    df['mrn'].map(liver_dict)                  # Liver first
    .combine_first(df['mrn'].map(recip_dict))       # then Recipient
    .combine_first(df['mrn'].map(donor_dict))       # then Donor
)

# Optional: Flag rows that still have no transplant date
# (uncomment when you're ready to implement review flagging)
# df.loc[df['transplant_date'].isna(), 'need_review'] = 'yes'

print("transplant_date column updated.")
print(f"Number of rows with transplant_date filled: {df['transplant_date'].notna().sum()}")

transplant_date column updated.
Number of rows with transplant_date filled: 1577


In [366]:
# fill patient_type: if visit_date - transplant_date is minus, zero or the transplant_date is NaN -> pre
# if the visit_date - transplant_date. patient_type -> post

# Ensure both columns are datetime (safe, coercive)
visit_dt = pd.to_datetime(df["visit_datetime"], errors="coerce")
tx_dt = pd.to_datetime(df["transplant_date"], errors="coerce")

# Calculate day difference
df["days"] = (visit_dt - tx_dt).dt.days

# Overwrite patient_type based on rules
df.loc[tx_dt.isna(), "patient_type"] = "pre"
df.loc[tx_dt.notna() & (df["days"] <= 0), "patient_type"] = "pre"
df.loc[tx_dt.notna() & (df["days"] > 0), "patient_type"] = "post"


In [367]:
df['r_d'].value_counts()

r_d
Post LD Tx                                        911
Liver Recipient                                   450
Pre Tx on workups                                 266
Post living donor nephrectomy                     158
Liver Donor                                       122
Post DD Tx                                         98
Waiting for first appointment                      50
Listed on DDWL                                     40
Pre Tx Ready for committee                         33
Pre Tx Discussed & ready for donor nephrectomy     30
Nephrology patient                                 21
Initial visit done, No LD, suspended for DDWL      20
Pre Tx Discussed & Ready for LD Tx                 18
Listed on DDWL & Ready for LD Tx                   13
Pre Tx Discussed with pending issues                9
Pre Tx on hold                                      9
Listed on DDWL - Inactive                           3
Name: count, dtype: int64

In [368]:
df[df['r_d'] == 'Nephrology patient']

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,need_review,r_d,patient_type,transplant_date,days
27,176279,RADHI NASER HASAN ALHODAR,1946-05-31,2025-12-01,VV,STARTED,NORMAL BOOKING,Clinical Nutrition,Ebtisam Hassan Husain Al-Ghawi,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,pre,NaT,NaN
102,201292,ALI IBRAHIM MOBARAK ABU ABSI,1968-09-23,2025-12-02,VV,STARTED,NORMAL BOOKING,Clinical Nutrition,Ebtisam Hassan Husain Al-Ghawi,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,pre,NaT,NaN
312,156996,RANA ABDULLAH A ALHAMAM,1975-09-17,2025-12-04,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Nihad Ali Al_Ahmed,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,pre,NaT,NaN
333,196863,MONEERAH NASSER SAAD ALSHAMMARI,1956-02-12,2025-12-30,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Nihad Ali Al_Ahmed,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,pre,NaT,NaN
361,166481,ZAINAB ALI HUSSAIN AL NASR,1953-03-16,2025-12-02,VV,STARTED,NORMAL BOOKING,Clinical Nutrition,Nihad Ali Al_Ahmed,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,pre,NaT,NaN
634,207593,SUWAID MESFER SWAID ALDOSSARI,1949-04-29,2025-12-08,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Fatima Abdullah Ali Al-Sinan,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,pre,NaT,NaN
637,55000,ZYAD YOSAF SALEH ALDUHAHI,2005-06-18,2025-12-10,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Fatima Abdullah Ali Al-Sinan,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,pre,NaT,NaN
649,13447,NARJES ABDULALI M ALSINAN,1956-02-12,2025-12-03,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Fatima Abdullah Ali Al-Sinan,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,pre,NaT,NaN
698,115829,ZAHRA MOHAMMED AHMED ALWABARI,2016-05-09,2025-12-01,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,pre,NaT,NaN
714,168247,SEDRA FARAJ MOHAMMED ALANAZI,2021-01-26,2025-12-15,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Fatimah Mansoor Al Faraj,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,pre,NaT,NaN


In [369]:
df['liver_status'].value_counts()

liver_status
Post Tx                        366
Hepatology patient             111
Pre Tx on workups               82
New patient                     47
Pre Tx ready for transplant     24
OPD Hepatology patient          16
Pre Tx on DDWL                  16
Pre Tx Ready for transplant      6
Hepatitis C patient              2
Pre Tx on hold                   2
Name: count, dtype: int64

In [370]:
df['patient_type'].value_counts()

patient_type
post                 1391
pre                   881
Post Tx                17
Post LD Tx             11
Post DD Tx              2
Pre Tx on workups       1
Name: count, dtype: int64

In [371]:
df.head(50)

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,need_review,r_d,patient_type,transplant_date,days
7,174153,AYSH ABDO HAMAD HADIDI,1955-02-23,2025-12-23,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,SUMAYAH ABDULAZIZ MOHAMMED ASKANDARANI,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,post,2025-11-26,27.0
9,77521,AMAL SULAIMAN OUDAH ALHAWITI,1986-09-28,2025-12-17,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,Post DD Tx,NaN,no,Post DD Tx,post,2016-04-24,3524.0
12,104431,SAIF RAJA H ALHAJRI,1976-06-28,2025-12-10,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,Recipient,Hepatology patient,Post DD Tx,NaN,no,NaN,post,2020-11-07,1859.0
13,101282,NOUF SAUD MOHAMMED ALDAWSARI,1982-11-29,2025-12-14,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,post,2022-01-08,1436.0
17,6364,MOHAMMED BEN ALI BIN SULAIMAN AL JAFAR,1982-08-20,2025-12-21,FOLLOW UP,STARTED,NORMAL BOOKING,TRANSPLANT SURGERY,MAHMOUD ALI MAHMOUD OBEID,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,post,2007-06-06,6773.0
18,66898,AMNA AHMED ABDULGAFOR AL SHEIKH,1984-01-04,2025-12-10,FOLLOW UP,ARRIVED,NORMAL BOOKING,TRANSPLANT SURGERY,MAHMOUD ALI MAHMOUD OBEID,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,post,2012-08-27,4853.0
19,208799,FARZANAH SAEED A ALI,1963-07-01,2025-12-15,FOLLOW UP,STARTED,NORMAL BOOKING,TRANSPLANT SURGERY,MAHMOUD ALI MAHMOUD OBEID,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,post,2025-11-24,21.0
20,220797,BASHAYER FAWAZ ALSAEGR,1992-04-17,2025-12-10,Walk-In,STARTED,WALK-IN,TRANSPLANT SURGERY,Mohammed Saad Ali AL-Qahtani,Donor,Pre Tx ready for transplant,NaN,NaN,no,Liver Donor,pre,NaT,NaN
21,220503,ALHASSAN ABDULRAHIM ABDULLAH ALZAHRANI,1999-08-20,2025-12-10,Walk-In,STARTED,WALK-IN,TRANSPLANT SURGERY,Mohammed Saad Ali AL-Qahtani,Donor,Post Tx,NaN,NaN,no,Liver Donor,pre,2025-12-16,-6.0
22,220468,SAAD SALEM M ALMAHRI,1986-03-17,2025-12-10,Walk-In,STARTED,WALK-IN,TRANSPLANT SURGERY,Mohammed Saad Ali AL-Qahtani,Donor,Post Tx,NaN,NaN,no,Liver Donor,pre,2025-12-15,-5.0


In [372]:
# Final patient_type classification logic
#
# This step overwrites patient_type (which initially contains only "pre" / "post")
# and assigns a final clinical category using transplant context and liver status.
#
# Priority order (top to bottom – first match wins):
#
# 1) Hepatology override:
#    If liver_status contains any of:
#       - "hepatology"
#       - "hepatitis"
#       - "new patients"
#    → patient_type = "Hepatology"
#
# 2) Liver transplant context:
#    If r_d contains "liver":
#       - patient_type == "post" → "Post-Liver Tx"
#       - patient_type == "pre"  → "Pre-Liver Tx"
#
# 3) Nephrology (non-transplant):
#    If r_d contains "nephrology":
#       → patient_type = "Nephrology"
#
# 4) Kidney transplant context:
#    If r_d is present AND does NOT contain "liver" or "nephrology":
#       - patient_type == "post" → "Post-Kidney Tx"
#       - patient_type == "pre"  → "Pre-Kidney Tx"
#
# 5) Final fallback:
#    Any remaining rows → "Pre-Kidney Tx"
#
# Notes:
# - Matching is case-insensitive
# - Rules are mutually exclusive and applied in strict priority order
# - This mirrors the quality department’s manual classification logic



import numpy as np


# Prepare case-insensitive conditions
r_d_lower = df['r_d'].fillna('').str.lower()
liver_status_lower = df['liver_status'].fillna('').str.lower()

# ── Priority conditions (applied top → bottom) ───────────────────────────────

# Priority 1: Hepatology override (absolute)
is_hepatology = liver_status_lower.str.contains('hepatology|hepatitis|new patients', na=False)

# Priority 2: Liver transplant context
is_liver_context = r_d_lower.str.contains('liver', na=False)
is_post = df['patient_type'] == 'post'
is_pre  = df['patient_type'] == 'pre'

# Priority 3: Nephrology (non-transplant)
is_nephrology = r_d_lower.str.contains('nephrology', na=False)

# Priority 4: Kidney transplant context
# (has r_d value, but not liver & not nephrology)
is_kidney_context = (
    df['r_d'].notna() &
    ~is_liver_context &
    ~is_nephrology
)

# ── Apply rules in strict priority order using np.select ─────────────────────

conditions = [
    is_hepatology,                                      # 1. Hepatology override
    is_liver_context & is_post,                         # 2. Post-Liver Tx
    is_liver_context & is_pre,                          # 2. Pre-Liver Tx
    is_nephrology,                                      # 3. Nephrology
    is_kidney_context & is_post,                        # 4. Post-Kidney Tx
    is_kidney_context & is_pre,                         # 4. Pre-Kidney Tx
]

choices = [
    "Hepatology",
    "Post-Liver Tx",
    "Pre-Liver Tx",
    "Nephrology",
    "Post-Kidney Tx",
    "Pre-Kidney Tx",
]

df['patient_type'] = np.select(conditions, choices, default="Pre-Kidney Tx")



In [373]:
df['patient_type'].value_counts()

patient_type
Post-Kidney Tx    1030
Pre-Kidney Tx      630
Post-Liver Tx      286
Pre-Liver Tx       207
Hepatology         129
Nephrology          21
Name: count, dtype: int64

In [374]:
df['clinic_name'].value_counts()

clinic_name
Adult Post Renal Transplant                    500
TRANSPLANT SURGERY                             458
Adult Post Kidney Transplant Virtual Clinic    264
Adult Pre Renal Transplant                     133
ADULT POST LIVER TRANSPLANT                    131
Anesthesia                                     130
Social Work                                    110
Pediatric Post Renal Transplant                102
Clinical Nutrition                              98
Adult Pre Liver Transplant                      97
Pediatric Liver Transplant                      66
Transplant Dressing (procedure)                 54
Psychology                                      47
Adult Post Liver Transplant Virtual             42
Adult Pre Renal Transplant Virtual Clinic       37
Pre anesthesia liver transplant                 15
Pediatric Pre Renal Transplant                  11
Adult Pre Liver Transplant Virtual               5
Transplant Cystoscopy Clinic                     3
Name: count, dtype:

In [375]:
df.head()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,need_review,r_d,patient_type,transplant_date,days
7,174153,AYSH ABDO HAMAD HADIDI,1955-02-23,2025-12-23,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,SUMAYAH ABDULAZIZ MOHAMMED ASKANDARANI,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,Post-Kidney Tx,2025-11-26,27.0
9,77521,AMAL SULAIMAN OUDAH ALHAWITI,1986-09-28,2025-12-17,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,Post DD Tx,NaN,no,Post DD Tx,Post-Kidney Tx,2016-04-24,3524.0
12,104431,SAIF RAJA H ALHAJRI,1976-06-28,2025-12-10,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,Recipient,Hepatology patient,Post DD Tx,NaN,no,NaN,Hepatology,2020-11-07,1859.0
13,101282,NOUF SAUD MOHAMMED ALDAWSARI,1982-11-29,2025-12-14,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,Post-Kidney Tx,2022-01-08,1436.0
17,6364,MOHAMMED BEN ALI BIN SULAIMAN AL JAFAR,1982-08-20,2025-12-21,FOLLOW UP,STARTED,NORMAL BOOKING,TRANSPLANT SURGERY,MAHMOUD ALI MAHMOUD OBEID,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,Post-Kidney Tx,2007-06-06,6773.0


In [376]:
# Create lookup dictionaries: MRN → Age Group

liver_age_dict = (
    liver_df[['r_mrn', 'age_group']]
    .dropna(subset=['r_mrn'])
    .set_index('r_mrn')['age_group']
    .to_dict()
)

recip_age_dict = (
    kidney_rec_df[['r_mrn', 'age_group']]
    .dropna(subset=['r_mrn'])
    .set_index('r_mrn')['age_group']
    .to_dict()
)

donor_age_dict = (
    kidney_donor_df[['d_mrn', 'age_group_d']]
    .dropna(subset=['d_mrn'])
    .set_index('d_mrn')['age_group_d']
    .to_dict()
)

# Populate age_group with priority: Liver → Recipient → Donor
df['age_group'] = (
    df['mrn'].map(liver_age_dict)                # 1. Liver first
    .combine_first(df['mrn'].map(recip_age_dict))     # 2. Kidney Recipient
    .combine_first(df['mrn'].map(donor_age_dict))     # 3. Kidney Donor
)



In [377]:
df['age_group'].value_counts()

age_group
Adult        2027
Pediatric     267
Name: count, dtype: int64

In [378]:
df.head()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,need_review,r_d,patient_type,transplant_date,days,age_group
7,174153,AYSH ABDO HAMAD HADIDI,1955-02-23,2025-12-23,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,SUMAYAH ABDULAZIZ MOHAMMED ASKANDARANI,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,Post-Kidney Tx,2025-11-26,27.0,Adult
9,77521,AMAL SULAIMAN OUDAH ALHAWITI,1986-09-28,2025-12-17,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,Post DD Tx,NaN,no,Post DD Tx,Post-Kidney Tx,2016-04-24,3524.0,Adult
12,104431,SAIF RAJA H ALHAJRI,1976-06-28,2025-12-10,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,Recipient,Hepatology patient,Post DD Tx,NaN,no,NaN,Hepatology,2020-11-07,1859.0,Adult
13,101282,NOUF SAUD MOHAMMED ALDAWSARI,1982-11-29,2025-12-14,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,Post-Kidney Tx,2022-01-08,1436.0,Adult
17,6364,MOHAMMED BEN ALI BIN SULAIMAN AL JAFAR,1982-08-20,2025-12-21,FOLLOW UP,STARTED,NORMAL BOOKING,TRANSPLANT SURGERY,MAHMOUD ALI MAHMOUD OBEID,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,Post-Kidney Tx,2007-06-06,6773.0,Adult


In [379]:
df['clinic_name'].value_counts()

clinic_name
Adult Post Renal Transplant                    500
TRANSPLANT SURGERY                             458
Adult Post Kidney Transplant Virtual Clinic    264
Adult Pre Renal Transplant                     133
ADULT POST LIVER TRANSPLANT                    131
Anesthesia                                     130
Social Work                                    110
Pediatric Post Renal Transplant                102
Clinical Nutrition                              98
Adult Pre Liver Transplant                      97
Pediatric Liver Transplant                      66
Transplant Dressing (procedure)                 54
Psychology                                      47
Adult Post Liver Transplant Virtual             42
Adult Pre Renal Transplant Virtual Clinic       37
Pre anesthesia liver transplant                 15
Pediatric Pre Renal Transplant                  11
Adult Pre Liver Transplant Virtual               5
Transplant Cystoscopy Clinic                     3
Name: count, dtype:

In [380]:
df['patient_type'].value_counts()

patient_type
Post-Kidney Tx    1030
Pre-Kidney Tx      630
Post-Liver Tx      286
Pre-Liver Tx       207
Hepatology         129
Nephrology          21
Name: count, dtype: int64

In [381]:
df['r_d'].value_counts()

r_d
Post LD Tx                                        911
Liver Recipient                                   450
Pre Tx on workups                                 266
Post living donor nephrectomy                     158
Liver Donor                                       122
Post DD Tx                                         98
Waiting for first appointment                      50
Listed on DDWL                                     40
Pre Tx Ready for committee                         33
Pre Tx Discussed & ready for donor nephrectomy     30
Nephrology patient                                 21
Initial visit done, No LD, suspended for DDWL      20
Pre Tx Discussed & Ready for LD Tx                 18
Listed on DDWL & Ready for LD Tx                   13
Pre Tx Discussed with pending issues                9
Pre Tx on hold                                      9
Listed on DDWL - Inactive                           3
Name: count, dtype: int64

In [382]:
df.head()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,need_review,r_d,patient_type,transplant_date,days,age_group
7,174153,AYSH ABDO HAMAD HADIDI,1955-02-23,2025-12-23,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,SUMAYAH ABDULAZIZ MOHAMMED ASKANDARANI,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,Post-Kidney Tx,2025-11-26,27.0,Adult
9,77521,AMAL SULAIMAN OUDAH ALHAWITI,1986-09-28,2025-12-17,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,Post DD Tx,NaN,no,Post DD Tx,Post-Kidney Tx,2016-04-24,3524.0,Adult
12,104431,SAIF RAJA H ALHAJRI,1976-06-28,2025-12-10,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,Recipient,Hepatology patient,Post DD Tx,NaN,no,NaN,Hepatology,2020-11-07,1859.0,Adult
13,101282,NOUF SAUD MOHAMMED ALDAWSARI,1982-11-29,2025-12-14,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,Post-Kidney Tx,2022-01-08,1436.0,Adult
17,6364,MOHAMMED BEN ALI BIN SULAIMAN AL JAFAR,1982-08-20,2025-12-21,FOLLOW UP,STARTED,NORMAL BOOKING,TRANSPLANT SURGERY,MAHMOUD ALI MAHMOUD OBEID,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,Post-Kidney Tx,2007-06-06,6773.0,Adult


In [383]:
df['r_d'].value_counts()

r_d
Post LD Tx                                        911
Liver Recipient                                   450
Pre Tx on workups                                 266
Post living donor nephrectomy                     158
Liver Donor                                       122
Post DD Tx                                         98
Waiting for first appointment                      50
Listed on DDWL                                     40
Pre Tx Ready for committee                         33
Pre Tx Discussed & ready for donor nephrectomy     30
Nephrology patient                                 21
Initial visit done, No LD, suspended for DDWL      20
Pre Tx Discussed & Ready for LD Tx                 18
Listed on DDWL & Ready for LD Tx                   13
Pre Tx Discussed with pending issues                9
Pre Tx on hold                                      9
Listed on DDWL - Inactive                           3
Name: count, dtype: int64

In [384]:
df.head()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,need_review,r_d,patient_type,transplant_date,days,age_group
7,174153,AYSH ABDO HAMAD HADIDI,1955-02-23,2025-12-23,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,SUMAYAH ABDULAZIZ MOHAMMED ASKANDARANI,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,Post-Kidney Tx,2025-11-26,27.0,Adult
9,77521,AMAL SULAIMAN OUDAH ALHAWITI,1986-09-28,2025-12-17,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,Post DD Tx,NaN,no,Post DD Tx,Post-Kidney Tx,2016-04-24,3524.0,Adult
12,104431,SAIF RAJA H ALHAJRI,1976-06-28,2025-12-10,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,Recipient,Hepatology patient,Post DD Tx,NaN,no,NaN,Hepatology,2020-11-07,1859.0,Adult
13,101282,NOUF SAUD MOHAMMED ALDAWSARI,1982-11-29,2025-12-14,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,Post-Kidney Tx,2022-01-08,1436.0,Adult
17,6364,MOHAMMED BEN ALI BIN SULAIMAN AL JAFAR,1982-08-20,2025-12-21,FOLLOW UP,STARTED,NORMAL BOOKING,TRANSPLANT SURGERY,MAHMOUD ALI MAHMOUD OBEID,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,Post-Kidney Tx,2007-06-06,6773.0,Adult


In [385]:
df[df['patient_type'] == 'Hepatology'][30:50]

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,need_review,r_d,patient_type,transplant_date,days,age_group
1181,189072,JABER YAHIA SALEM ALMALKI,1969-04-16,2025-12-15,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Pre Liver Transplant,Eyad Mohammad Osman Elhassan Gadour,Recipient,Hepatology patient,Post LD Tx,NaN,no,Liver Recipient,Hepatology,2025-11-06,39.0,Adult
1224,221032,Basmah Abdulrazaq Abdulrahman,1993-01-01,2025-12-09,NEW,STARTED,NORMAL BOOKING,Adult Pre Liver Transplant,NOORA HASSAN B AL FARAJ,Recipient,Hepatology patient,NaN,NaN,no,Liver Recipient,Hepatology,NaT,NaN,Adult
1230,195166,HASSAN MAHMOOD OMAR HAWSAWI,1962-10-05,2025-12-02,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Pre Liver Transplant,NOORA HASSAN B AL FARAJ,Recipient,Hepatology patient,NaN,NaN,no,Liver Recipient,Hepatology,NaT,NaN,Adult
1237,176215,Fatimah Saeed Abdullah Al Baggal,1992-02-16,2025-12-22,NEW,STARTED,NORMAL BOOKING,Adult Pre Liver Transplant,Eyad Mohammad Osman Elhassan Gadour,Recipient,Hepatology patient,NaN,NaN,no,Liver Recipient,Hepatology,NaT,NaN,Adult
1239,220244,KHULUD ABDULLAH DHAFER ALOTAIBI,1985-03-22,2025-12-15,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Pre Liver Transplant,Eyad Mohammad Osman Elhassan Gadour,Recipient,Hepatology patient,NaN,NaN,no,Liver Recipient,Hepatology,NaT,NaN,Adult
1331,218286,ABDULKAREEM ZAYED ALI ALWASHI,2000-01-01,2025-12-24,Walk-In,STARTED,WALK-IN,Adult Post Renal Transplant,MAHER MOHAMMED RAFAT ALDEMERDASH,Recipient,Hepatology patient,Post LD Tx,NaN,no,Post LD Tx,Hepatology,2025-11-17,37.0,Adult
1370,122652,RAWAN AHMED NASSAR ALATAWI,2007-04-03,2025-12-08,CHART CHECK,STARTED,WALK-IN,Pediatric Post Renal Transplant,ABDULAZIZ AHMED SALEM BA MHRAZ,Recipient,Hepatology patient,Post LD Tx,NaN,no,Post LD Tx,Hepatology,2025-11-25,13.0,Pediatric
1383,120889,AMMAR DHAFER SAAD ALALYANI,2015-11-26,2025-12-11,CHART CHECK,STARTED,WALK-IN,Pediatric Post Renal Transplant,Mohammed Abdulmoatani Almaghrabi,Recipient,Hepatology patient,Post LD Tx,NaN,no,Post LD Tx,Hepatology,2019-04-29,2418.0,Pediatric
1393,52498,RAWAF AHMED YOUSEF ALJALAL,2010-03-25,2025-12-29,NEW,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,ABDULAZIZ AHMED SALEM BA MHRAZ,Recipient,Hepatology patient,Post LD Tx,NaN,no,Post LD Tx,Hepatology,2024-12-24,370.0,Pediatric
1395,53033,ANWAR MOHAMMED ABDULLAH ALSWAILMI,2007-11-21,2025-12-07,CHART CHECK,STARTED,WALK-IN,Pediatric Post Renal Transplant,ABDULAZIZ AHMED SALEM BA MHRAZ,Recipient,Hepatology patient,Post DD Tx,NaN,no,Post DD Tx,Hepatology,2011-12-05,5116.0,Pediatric


In [386]:

# Final logic for correcting / filling r_d column
# Rules applied in this strict top-to-bottom priority order:

# 1. If patient_type == "Hepatology"
#    → r_d = "Liver Recipient"

# 2. If patient_type == "Nephrology"
#    → r_d = "Kidney Recipient"

# 3. If patient_type in ["Post-Liver Tx", "Pre-Liver Tx"]
#    → keep the current r_d value as-is 
#      (it should already be either "Liver Recipient" or "Liver Donor")

# 4. If patient_type in ["Post-Kidney Tx", "Pre-Kidney Tx"]
#    → Decision order:
#      - If kidney_recipient is NOT NULL → r_d = "Kidney Recipient"
#      - Else if kidney_donor is NOT NULL   → r_d = "Kidney Donor"
#      - Else (both kidney_recipient AND kidney_donor are NULL)
#        → keep current r_d value unchanged
#        → AND set need_review = "yes" (change from default "no")

# All other cases: keep current r_d (though in practice should be covered above)

# Notes:
# - Liver cases trust existing r_d (already cleaned correctly)
# - Kidney cases prefer recipient status when available
# - Only flag for manual review when there's true ambiguity 
#   (no clear kidney role in source registry columns)


# 1. Hepatology & Nephrology overrides
hepatology_mask = df['patient_type'] == "Hepatology"
nephrology_mask = df['patient_type'] == "Nephrology"

df.loc[hepatology_mask, 'r_d'] = "Liver Recipient"
df.loc[nephrology_mask, 'r_d'] = "Kidney Recipient"

# 2. Liver cases - keep current r_d (no change needed)

# 3. Kidney transplant cases (Post & Pre)
kidney_tx_mask = df['patient_type'].isin(["Post-Kidney Tx", "Pre-Kidney Tx"])

# Sub-conditions for kidney cases
has_recipient = df['kidney_recipient'].notna()
has_donor     = df['kidney_donor'].notna()

# Apply preference: Recipient > Donor > keep current
df.loc[kidney_tx_mask & has_recipient, 'r_d'] = "Kidney Recipient"
df.loc[kidney_tx_mask & ~has_recipient & has_donor, 'r_d'] = "Kidney Donor"

# 4. Ambiguous kidney cases → flag for review
ambiguous_kidney = (
    kidney_tx_mask & 
    ~has_recipient & 
    ~has_donor
)

df.loc[ambiguous_kidney, 'need_review'] = "yes"


In [387]:
df['clinic_name'].value_counts()

clinic_name
Adult Post Renal Transplant                    500
TRANSPLANT SURGERY                             458
Adult Post Kidney Transplant Virtual Clinic    264
Adult Pre Renal Transplant                     133
ADULT POST LIVER TRANSPLANT                    131
Anesthesia                                     130
Social Work                                    110
Pediatric Post Renal Transplant                102
Clinical Nutrition                              98
Adult Pre Liver Transplant                      97
Pediatric Liver Transplant                      66
Transplant Dressing (procedure)                 54
Psychology                                      47
Adult Post Liver Transplant Virtual             42
Adult Pre Renal Transplant Virtual Clinic       37
Pre anesthesia liver transplant                 15
Pediatric Pre Renal Transplant                  11
Adult Pre Liver Transplant Virtual               5
Transplant Cystoscopy Clinic                     3
Name: count, dtype:

In [317]:
df['r_d'].value_counts()

r_d
Kidney Recipient    1265
Liver Recipient      516
Kidney Donor         391
Liver Donor          122
Name: count, dtype: int64

In [388]:
# Logic for creating the new 'clinic' column
# Rules are applied in order - first matching condition wins (top-to-bottom priority)

# All string matching is:
# - case-insensitive
# - partial contains (anywhere in the clinic_name string)

# 1. If clinic_name contains "renal" or "kidney"
#    → clinic = "Kidney Tx"

# 2. Else if clinic_name contains any of:
#    - "pediatric liver transplant"
#    - "adult pre liver transplant"
#    - "adult post liver transplant"
#    → clinic = "liver Tx"

# 3. Else if clinic_name contains "anesthesia"
#    → Check r_d (case-insensitive contains):
#      - if r_d contains "kidney" → clinic = "Anesthesia-Kidney"
#      - if r_d contains "liver"  → clinic = "Anesthesia-Liver"
#      - if neither → clinic remains NaN

# 4. Else if clinic_name contains "transplant urology" or "transplant cystoscopy clinic"
#    → clinic = "Urology"

# 5. Else if clinic_name contains "clinical nutrition"
#    → clinic = "Nutrition"

# 6. Else if clinic_name contains "transplant dressing (procedure)"
#    → clinic = "Surgery-Dressing"

# 7. Else if clinic_name contains "transplant surgery"
#    → Check r_d (case-insensitive contains):
#      - if r_d contains "kidney" → clinic = "Surgery-Kidney"
#      - else → clinic = "Surgery-Liver"

# 8. Else if clinic_name contains "psychology"
#    → clinic = "Psychology"

# 9. Else if clinic_name contains "social work"
#    → clinic = "Social Work"

# 10. Fallback (no match above)
#     → clinic remains NaN



# Make sure clinic_name is string and handle NaN
df['clinic_name'] = df['clinic_name'].astype(str).fillna('')

# Prepare lowercase versions for matching
clinic_lower = df['clinic_name'].str.lower()
rd_lower     = df['r_d'].fillna('').str.lower()

# Initialize the new column
df['clinic'] = pd.NA

# 1. Kidney Tx (renal or kidney)
mask_kidney = clinic_lower.str.contains('renal|kidney', na=False)
df.loc[mask_kidney, 'clinic'] = 'Kidney Tx'

# 2. Liver Tx (specific phrases)
mask_liver_specific = clinic_lower.str.contains(
    'pediatric liver transplant|adult pre liver transplant|adult post liver transplant',
    na=False
)
df.loc[mask_liver_specific & df['clinic'].isna(), 'clinic'] = 'liver Tx'

# 3. Anesthesia cases
mask_anesthesia = clinic_lower.str.contains('anesthesia', na=False)
df.loc[mask_anesthesia & df['clinic'].isna(), 'clinic'] = pd.NA  # default

df.loc[
    mask_anesthesia & df['clinic'].isna() & rd_lower.str.contains('kidney'),
    'clinic'
] = 'Anesthesia-Kidney'

df.loc[
    mask_anesthesia & df['clinic'].isna() & rd_lower.str.contains('liver'),
    'clinic'
] = 'Anesthesia-Liver'

# 4. Urology
mask_urology = clinic_lower.str.contains('transplant urology|transplant cystoscopy clinic', na=False)
df.loc[mask_urology & df['clinic'].isna(), 'clinic'] = 'Urology'

# 5. Nutrition
mask_nutrition = clinic_lower.str.contains('clinical nutrition', na=False)
df.loc[mask_nutrition & df['clinic'].isna(), 'clinic'] = 'Nutrition'

# 6. Surgery Dressing
mask_dressing = clinic_lower.str.contains('transplant dressing (procedure)', na=False)
df.loc[mask_dressing & df['clinic'].isna(), 'clinic'] = 'Surgery-Dressing'

# 7. Transplant Surgery
mask_surgery = clinic_lower.str.contains('transplant surgery', na=False)
df.loc[mask_surgery & df['clinic'].isna(), 'clinic'] = 'Surgery-Liver'  # default for surgery

df.loc[
    mask_surgery & df['clinic'].isna() & rd_lower.str.contains('kidney'),
    'clinic'
] = 'Surgery-Kidney'

# 8. Psychology
mask_psych = clinic_lower.str.contains('psychology', na=False)
df.loc[mask_psych & df['clinic'].isna(), 'clinic'] = 'Psychology'

# 9. Social Work
mask_social = clinic_lower.str.contains('social work', na=False)
df.loc[mask_social & df['clinic'].isna(), 'clinic'] = 'Social Work'



/var/folders/67/spj008b90s1d1nmg1kfhfrzw0000gn/T/ipykernel_83722/3488552905.py:92: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_dressing = clinic_lower.str.contains('transplant dressing (procedure)', na=False)
